In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import json
import os.path as osp
import warnings
from glob import glob
from typing import Callable, Literal, Optional

import cv2
import lpips
import mediapy as media
import numpy as np
import torch
from tqdm import tqdm

/home/hangg/.anaconda3/envs/nerfacc/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
LPIPS_MODELS = {"alex": None, "vgg": None}


def get_compute_lpips(
    net: Literal["alex", "vgg"] = "vgg", device: str = "cuda"
) -> Callable:
    """Get the LPIPS metric function as a singleton to avoid extra copy."""

    global LPIPS_MODELS
    if LPIPS_MODELS[net] is None:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            LPIPS_MODELS[net] = lpips.LPIPS(net=net, spatial=True).to(device)
    model = LPIPS_MODELS[net]

    @torch.inference_mode()
    def compute_lpips(
        img0: torch.Tensor,
        img1: torch.Tensor,
    ) -> np.array:
        """Compute LPIPS between two images.

        Args:
            img0: (H, W, 3) in uint8.
            img1: (H, W, 3).

        Returns:
            lpips: (), LPIPS score in [0, 1].
        """
        img0 = (
            torch.from_numpy(img0 / 255.0)
            .to(dtype=torch.float32, device=device)
            .movedim(-1, 0)[None]
        )
        img1 = (
            torch.from_numpy(img1 / 255.0)
            .to(dtype=torch.float32, device=device)
            .movedim(-1, 0)[None]
        )
        return model(img0, img1)[0, 0, ..., None].mean().item()

    return compute_lpips

## Blender

In [4]:
compute_lpips = get_compute_lpips(net="vgg")

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: /home/hangg/.anaconda3/envs/nerfacc/lib/python3.9/site-packages/lpips/weights/v0.1/vgg.pth


In [12]:
SCENES = [
    "chair",
    "drums",
    "ficus",
    "hotdog",
    "lego",
    "materials",
    "mic",
    "ship",
]

In [13]:
# TensoRF
for scene in SCENES:
    scores = []
    for i in tqdm(
        range(
            len(
                [
                    p
                    for p in glob(
                        f"/shared/hangg/projects/tensorf/data/nerf_synthetic/{scene}/test/*.png"
                    )
                    if "depth" not in p and "normal" not in p
                ]
            )
        )
    ):
        img = (
            cv2.imread(
                f"/shared/hangg/projects/tensorf/data/nerf_synthetic/{scene}/test/r_{i:d}.png",
                flags=cv2.IMREAD_UNCHANGED,
            )
            / 255.0
        )
        img = (
            (img[..., :3] * img[..., -1:] + 1 - img[..., -1:]) * 255
        ).astype(np.uint8)
        img = img[..., ::-1]
        img_pred = cv2.imread(
            f"/shared/hangg/projects/tensorf/log/tensorf_{scene}_VM_15k_nerfacc/tensorf_{scene}_VM_15k_nerfacc/imgs_test_all/{i:03d}.png"
        )[..., ::-1]
        scores.append(compute_lpips(img, img_pred))
    print(scene, np.mean(scores))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:20<00:00,  9.75it/s]

hotdog 0.028047882309183477


## T&T

In [4]:
compute_lpips = get_compute_lpips(net="vgg")

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: /home/hangg/.anaconda3/envs/nerfacc/lib/python3.9/site-packages/lpips/weights/v0.1/vgg.pth


In [37]:
SCENES = [
    "Barn",
    # "Caterpillar",
    # "Family",
    # "Truck",
]

In [38]:
# TensoRF
for scene in SCENES:
    scores = []
    for i, path in enumerate(
        tqdm(
            sorted(
                glob(
                    f"/shared/hangg/projects/tensorf/data/TanksAndTemple/{scene}/rgb/1_*.png"
                )
            )
        )
    ):
        img = cv2.imread(path)[..., ::-1]
        scene = scene.lower()
        img_pred = cv2.imread(
            # f"/shared/hangg/projects/tensorf/log/tensorf_{scene}_VM_15k/tensorf_{scene}_VM_15k/imgs_test_all/{i:03d}.png"
            # f"/shared/hangg/projects/tensorf/log/tensorf_{scene}_VM_15k_nerfacc/tensorf_{scene}_VM_15k_nerfacc/imgs_test_all/{i:03d}.png"
            f"/shared/hangg/projects/tensorf/log/tensorf_{scene}_VM_15k_prop/imgs_test_all/{i:03d}.png"
        )[..., ::-1]
        scores.append(compute_lpips(img, img_pred))
    print(scene, np.mean(scores))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:13<00:00,  3.57it/s]

barn 0.24504011558989683


In [32]:
f"/shared/hangg/projects/tensorf/log/tensorf_{scene}_VM_15k_prop/tensorf_{scene}_VM_15k_prop/imgs_test_all/{i:03d}.png"

'/shared/hangg/projects/tensorf/log/tensorf_family_VM_15k_prop/tensorf_family_VM_15k_prop/imgs_test_all/000.png'

## Noisy Blender

In [79]:
compute_lpips = get_compute_lpips(net="alex")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [on]
Loading model from: /home/hangg/.anaconda3/envs/nerfacc/lib/python3.9/site-packages/lpips/weights/v0.1/alex.pth


In [98]:
SCENES = [
    "chair",
    "drums",
    "ficus",
    "hotdog",
    "lego",
    "materials",
    "mic",
    "ship",
]

In [99]:
# BARF
for scene in SCENES:
    scores = []
    paths = sorted(
        glob(
            f"/shared/hangg/projects/barf/output/{scene}/{scene}_nerfacc/test_view/rgb_GT_*.png"
        )
    )
    pred_paths = [p.replace("_GT_", "_") for p in paths]
    for path, pred_path in zip(tqdm(paths), pred_paths):
        img = (
            cv2.imread(
                path,
                flags=cv2.IMREAD_UNCHANGED,
            )
        )[..., ::-1]
        img_pred = cv2.imread(pred_path)[..., ::-1]
        scores.append(compute_lpips(img, img_pred))
    print(scene, np.mean(scores))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 51.30it/s]


chair 0.03373537153936922


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 48.17it/s]


drums 0.0707241720892489


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 45.37it/s]


...

materials 0.029524475745856763


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:03<00:00, 55.75it/s]


mic 0.03330276141874492


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:04<00:00, 43.68it/s]

ship 0.09396069411188364


## D-NeRF

In [44]:
compute_lpips = get_compute_lpips(net="vgg")

In [45]:
SCENES = [
    "bouncingballs",
    "hellwarrior",
    "hook",
    "jumpingjacks",
    "lego",
    "mutant",
    "standup",
    "trex",
]

In [39]:
# TiNeuVox
for scene in SCENES:
    scores = []
    for i in tqdm(
        range(
            len(
                glob(
                    f"/shared/hangg/projects/tineuvox/data/dnerf/{scene}/test/*.png"
                )
            )
        )
    ):
        img = (
            cv2.imread(
                f"/shared/hangg/projects/tineuvox/data/dnerf/{scene}/test/r_{i:03d}.png",
                flags=cv2.IMREAD_UNCHANGED,
            )
            / 255.0
        )
        img = (
            (img[..., :3] * img[..., -1:] + 1 - img[..., -1:]) * 255
        ).astype(np.uint8)
        img = cv2.resize(
            img,
            (400, 400),
            interpolation=cv2.INTER_AREA,
        )[..., ::-1]
        img_pred = cv2.imread(
            # f"/shared/hangg/projects/tineuvox/logs/nerf_synthetic/small/dnerf_{scene}-400_2/render_test_fine_last/{i:03d}.png"
            f"/shared/hangg/projects/tineuvox/logs/nerf_synthetic/small/dnerf_{scene}-400/render_test_fine_last/{i:03d}.png"
        )[..., ::-1]
        scores.append(compute_lpips(img, img_pred))
    print(scene, np.mean(scores))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 14.84it/s]


bouncingballs 0.027032664604485036


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 16.53it/s]


hellwarrior 0.06595230624079704


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 15.26it/s]


...

mutant 0.04209237983450294


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 19.97it/s]


standup 0.025279932329431175


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 17.65it/s]

trex 0.053174083679914476


In [47]:
SCENES = [
    # "bouncingballs",
    # "hellwarrior",
    # "hook",
    "jumpingjacks",
    # "lego",
    # "mutant",
    # "standup",
    # "trex",
]

In [48]:
# K-Planes
for scene in SCENES:
    scores = []
    video = media.read_video(
        f"/shared/hangg/projects/kplanes/logs/syntheticdynamic/{scene}_hybrid/step30000.mp4"
    )
    imgs = video[:, 800:1600]
    img_preds = video[:, :800]
    for img, img_pred in zip(tqdm(imgs), img_preds):
        scores.append(compute_lpips(img, img_pred))
    print(scene, np.mean(scores))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 19.46it/s]

jumpingjacks 0.041373722814023496


## HyperNeRF

In [4]:
compute_lpips = get_compute_lpips(net="vgg")

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: /home/hangg/.anaconda3/envs/nerfacc/lib/python3.9/site-packages/lpips/weights/v0.1/vgg.pth


In [101]:
SCENES = [
    # "hypernerf/vrig-3dprinter",
    # "nerfies/broom",
    # "hypernerf/vrig-chicken",
    "hypernerf/vrig-peel-banana",
]

In [102]:
# TiNeuVox
for scene in SCENES:
    with open(
        f"/shared/hangg/projects/tineuvox/data/{scene}/dataset.json"
    ) as f:
        test_names = json.load(f)["val_ids"]
    scores = []
    for i, test_name in enumerate(tqdm(test_names)):
        img = cv2.imread(
            f"/shared/hangg/projects/tineuvox/data/{scene}/rgb/2x/{test_name}.png",
            flags=cv2.IMREAD_UNCHANGED,
        )[..., ::-1]
        scene_suffix = scene.split("/")[-1].removeprefix("vrig-")
        img_pred = cv2.imread(
            f"/shared/hangg/projects/tineuvox/logs/vrig_data/vrig/base-{scene_suffix}_3/render_test_fine_last/{i:03d}.png"
        )[..., ::-1]
        scores.append(compute_lpips(img, img_pred))
    print(scene, np.mean(scores))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 513/513 [00:31<00:00, 16.36it/s]

hypernerf/vrig-peel-banana 0.30839472890016395
